In [83]:
!pip install --upgrade nltk gensim bokeh umap-learn

import itertools
import string

import numpy as np
import umap
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from matplotlib import pyplot as plt

from IPython.display import clear_output
import nltk
nltk.download('wordnet')
from IPython.display import clear_output

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [84]:
# выгружаем датасет:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc

File ‘./quora.txt’ already there; not retrieving.


In [91]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

In [92]:
tokenizer = WordPunctTokenizer()
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
filtered_vacab = []

In [97]:
data_lower = [i.lower() for i in data[:50]]
data_tok = [tokenizer.tokenize(i) for i in data_lower]

all_words = []
for i in data_tok:
    for j in i:
      all_words.append(j)
vocab = unique(all_words)

In [98]:
def vectorize(tokens):
    ''' This function takes list of words in a sentence as input 
    and returns a vector of size of filtered_vocab.It puts 0 if the 
    word is not present in tokens and count of token if present.'''
    vector=[]
    for w in filtered_vacab:
        vector.append(tokens.count(w))
    return vector
def unique(sequence):
    '''This functions returns a list in which the order remains 
    same and no item repeats.Using the set() function does not 
    preserve the original ordering,so i didnt use that instead'''
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

#create a list of stopwords.You can import stopwords from nltk too
stopwords=["to","is","a"]

#list of special characters.You can use regular expressions too
special_char=[",",":"," ",";",".","?"]


In [99]:
filtered_vacab = []
for w in vocab:
    if w not in stopwords and w not in special_char:
        filtered_vacab.append(w)

vector_of_data_tok = []
for i in data_tok:
    vector_of_data_tok.append(vectorize(i))




In [100]:
from numpy import dot
from numpy.linalg import norm


cos = dict()
for i, a in enumerate(vector_of_data_tok):
  for u, b in enumerate(vector_of_data_tok):
    if i != u:
      cos_sim = dot(a, b)/(norm(a)*norm(b))
      cos[(i,u)] = cos_sim
pair = None
for x in cos:
  if cos.get(x) == max(cos.values()):
    print(x)
    pair = x
    break
for x in pair:
  print(data_tok[x])

(8, 27)
['what', 'is', 'your', 'review', 'of', 'osquery', '?']
['what', 'is', 'your', 'review', 'of', 'ipad', 'mini', '2', '?']


In [75]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [77]:
text = data[:50]
count = CountVectorizer()
word_count=count.fit_transform(text)
print(word_count)

  (0, 37)	1
  (0, 101)	1
  (0, 26)	1
  (0, 265)	2
  (0, 161)	1
  (0, 82)	1
  (0, 80)	1
  (0, 233)	1
  (0, 210)	1
  (0, 131)	1
  (0, 190)	1
  (0, 12)	1
  (0, 110)	1
  (0, 25)	1
  (1, 257)	1
  (1, 14)	1
  (1, 218)	1
  (1, 252)	1
  (1, 236)	1
  (1, 182)	1
  (1, 88)	1
  (1, 93)	1
  (1, 6)	1
  (2, 261)	2
  (2, 256)	1
  :	:
  (47, 14)	1
  (47, 261)	1
  (47, 231)	1
  (47, 124)	1
  (47, 35)	1
  (47, 237)	1
  (47, 50)	1
  (47, 22)	1
  (48, 236)	1
  (48, 262)	1
  (48, 64)	1
  (48, 94)	1
  (48, 102)	1
  (48, 249)	1
  (48, 33)	1
  (48, 193)	1
  (49, 11)	1
  (49, 211)	1
  (49, 169)	1
  (49, 29)	1
  (49, 56)	1
  (49, 179)	1
  (49, 163)	1
  (49, 221)	1
  (49, 116)	1


In [78]:
print(word_count.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [79]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=count.get_feature_names(),columns=["idf_weights"])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [80]:
#inverse document frequency
df_idf.sort_values(by=['idf_weights'])

,idf_weights
the,2.041454
is,2.159237
how,2.159237
what,2.223775
can,2.446919
...,...
goldman,4.238678
graphic,4.238678
great,4.238678
guy,4.238678


In [81]:
#tfidf
tf_idf_vector=tfidf_transformer.transform(word_count)
feature_names = count.get_feature_names()
first_document_vector=tf_idf_vector[1]
df_tfifd= pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df_tfifd.sort_values(by=["tfidf"],ascending=False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,tfidf
fast,0.375678
ways,0.375678
addiction,0.375678
some,0.375678
food,0.375678
...,...
from,0.000000
george,0.000000
get,0.000000
girls,0.000000


In [82]:
vectors = tf_idf_vector.toarray()
cos = dict()
for i, a in enumerate(vectors):
  for u, b in enumerate(vectors):
    if i != u:
      cos_sim = dot(a, b)/(norm(a)*norm(b))
      cos[(i,u)] = cos_sim
pair = None
for x in cos:
  if cos.get(x) == max(cos.values()):
    print(x)
    pair = x
    break
for x in pair:
  print(text[x])

(8, 27)
What is your review of osquery?

What is your review of iPad Mini 2?

